This notebook helps test a local or pip installed copy of llmsherpa with the ingestor core code

In [1]:
!pip install llmsherpa

You should consider upgrading via the 'C:\Users\fabia\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Add llmsherpa
###  This is used to parse PDFs and allows for smart chunking based on the documents structure (chapters, sections, tables, paragraphs).

In [1]:
import os, sys
%load_ext autoreload
from llmsherpa.readers import LayoutPDFReader
from IPython.core.display import display, HTML
%autoreload 2

C:\Users\fabia\AppData\Local\Temp\ipykernel_27564\1052947477.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [245]:
directory_path = "/Users/fabia/Desktop/testapi"
sys.path.insert(0, directory_path)


llmsherpa_api_url = "http://localhost:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true"
# pdf_url = "data/et200sp_system_manual_en-US_en-US_stripped.pdf"
pdf_url = "data/test_manual_new.pdf"


do_ocr = True
if do_ocr:
    llmsherpa_api_url = llmsherpa_api_url + "&applyOcr=yes"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

## Complete parsed document:

In [246]:
# HTML(doc.sections()[0].to_html(include_children=True, recurse=True))
# doc.sections()[1].block_json
# doc.sections()[0].to_text()
# doc.sections()[1].bbox
# llmsherpa.readers.Layout
HTML(doc.to_html())

## Parsed Chunks

In [247]:
for chunk in doc.chunks():
    print(chunk.to_context_text())
    print("-"*80)

Installation
7
--------------------------------------------------------------------------------
Installation > 7.1 Basics > Introduction
All modules of the ET 200SP distributed I/O system are open equipment.
This means you may only install the ET 200SP distributed I/O system in housings, cabinets or electrical operating rooms and in a dry indoor environment (degree of protection IP20).
The housings, cabinets and electrical operating rooms must guarantee protection against electric shock and spread of fire.
The requirements regarding mechanical strength must also be met.
The housings, cabinets, and electrical operating rooms must not be accessible without a key or tool.
Personnel with access must have been trained or authorized.
--------------------------------------------------------------------------------
Installation > 7.1 Basics > Installation location
Install the ET 200SP distributed I/O system in a suitable enclosure/control cabinet with sufficient mechanical strength and fire pr

In [257]:
for chunk in doc.chunks():
    if chunk.parent.title == "Mounting rules for reducing the thermal load":
        print(chunk.to_context_text())
        print()
        print("-"*80)

        cutText = chunk.to_context_text().splitlines(True)[1:]  # splitlines(True) keeps the line breaks
        new_text = ''.join(cutText)
        print(new_text)

Installation > 7.1 Basics > Mounting rules for reducing the thermal load
The following rules reduce the thermal load of the ET 200SP distributed I/O system in the control cabinet:
• Separate 2 modules with high power dissipation with a module of low power dissipation or by an empty space.
• Mix modules with higher power dissipation and modules with less power dissipation.
For example, modules with 16 outputs have a higher power dissipation than modules with 8 outputs.
• You should give preference to the horizontal mounting position.
• For vertical mounting position, plug modules with high power dissipation at the top, the interface module/CPU at the bottom.
7.2 Installation conditions for motor starters
• Mount an ET 200SP station with modules with high power dissipation in the lower area of the control cabinet.
• For a multi-tier configuration, plug modules with high power dissipation on the sides so that the waste heat can rise to the top unhindered.
• Avoid air movements at the term

### Post process the parsed chunks
As the parsed chunks can be to graniular we merge chunks that belong to the same (sub)section. We also merge the corresponding metadata like section, page_idx, parent, ...

In [248]:
def get_chunk_parent_titles(chunk):
    # Build a string that consists of recursively getting the chunk parent titles
    parent_titles = ""
    parent = chunk.parent
    while parent:
        try:
            parent_titles = parent.title + " > " + parent_titles
            parent = parent.parent
        except Exception as e:
            break
    # Remove the trailing " >"
    parent_titles = parent_titles.rstrip(" >")
    return parent_titles

In [260]:
# We want the full context_text without the section title. The section title is included in the metadata

def remove_first_line(text):
    lines = text.splitlines(True)[1:]  # splitlines(True) keeps the line breaks
    return ''.join(lines)

beforeString = doc.chunks()[12].to_context_text()
print(beforeString)
print("-"*80)
afterString = remove_first_line(beforeString)
print(afterString)

Installation > 7.1 Basics > Mounting rail > NOTE
For increased vibration and shock loads, you can mount the ET 200SP system on the SIMATIC system rail.
--------------------------------------------------------------------------------
For increased vibration and shock loads, you can mount the ET 200SP system on the SIMATIC system rail.


In [263]:
# Dictionary to store merged content
merged_chunks = {}

# Iterate through each chunk and merge content
for chunk in doc.chunks():
    section = get_chunk_parent_titles(chunk)
    text = remove_first_line(chunk.to_context_text())
    page_idx = chunk.page_idx + 1 
    parent = chunk.parent.title
    level = chunk.level

    if section not in merged_chunks:
        merged_chunks[section] = {
            "text": text,
            "start_page": page_idx,
            "end_page": page_idx,
            "parent": parent,
            "level": level
        }
    else:
        merged_chunks[section]["text"] += "\n" + text
        merged_chunks[section]["end_page"] = max(merged_chunks[section]["end_page"], page_idx)

# Convert the merged dictionary back to list of chunks with updated page_nr
final_chunks = []
for section, data in merged_chunks.items():
    start_page = data["start_page"]
    end_page = data["end_page"]
    if start_page == end_page:
        page_nr = str(start_page) # If there is only one page set that as the page number
    else:
        page_nr = f"{start_page}-{end_page}" # If there are multiple pages set the range as the page number
    
    final_chunks.append({
        "section": section,
        "text": data["text"],
        "page_nr": page_nr,
        "parent": data["parent"],
        "level": data["level"]
    })

# Output the final chunks
for chunk in final_chunks:
    print(f"Section: {chunk['section']}")
    print(f"Page Nr: {chunk['page_nr']}")
    print(f"Section Parent: {chunk['parent']}")
    print(f"Level: {chunk['level']}")
    print(f"Text: {chunk['text']}")
    print("="*80)


Section: Installation
Page Nr: 1
Section Parent: Installation
Level: 0
Text: 7
Section: Installation > 7.1 Basics > Introduction
Page Nr: 1
Section Parent: Introduction
Level: 3
Text: All modules of the ET 200SP distributed I/O system are open equipment.
This means you may only install the ET 200SP distributed I/O system in housings, cabinets or electrical operating rooms and in a dry indoor environment (degree of protection IP20).
The housings, cabinets and electrical operating rooms must guarantee protection against electric shock and spread of fire.
The requirements regarding mechanical strength must also be met.
The housings, cabinets, and electrical operating rooms must not be accessible without a key or tool.
Personnel with access must have been trained or authorized.
Section: Installation > 7.1 Basics > Installation location
Page Nr: 1
Section Parent: Installation location
Level: 3
Text: Install the ET 200SP distributed I/O system in a suitable enclosure/control cabinet with suf

### Print chunk metadata

In [240]:
def print_chunk_metadata(chunk):
    # print("Full json: ", chunk.block_json)
    # print()
    print("Page Nr: ", chunk.page_idx + 1)
    print("Tag: ", chunk.tag)
    print("Parent Section: ", chunk.parent.title)
    print("Level in the hierarchy: ", chunk.level)
    print("Parent Section Hierarchy: ", get_chunk_parent_titles(chunk))
    # print("Text: ", chunk.to_text())
    print()
    print("Context Text: ", chunk.to_context_text())
    print()
    print("="*100)


chunk = doc.chunks()[26]
print_chunk_metadata(chunk)


Page Nr:  4
Tag:  para
Parent Section:  WARNING Protection from conductive contamination
Parent Section Hierarchy:  Installation > 7.1 Basics > General rules for installation > WARNING Protection from conductive contamination
Level in the hierarchy:  4

Context Text:  Installation > 7.1 Basics > General rules for installation > WARNING Protection from conductive contamination
This can be achieved, for example, by installing the devices in a control cabinet with the appropriate degree of protection.



## Parsed Tables

In [165]:
for table in doc.tables():
    print(table.to_text())

 | ① | BusAdapter
 | ② | Mounting rail
 | ③ | Reference identification label
 | ④ | CPU/interface module
 | ⑤ | Light-colored BaseUnit BUD with infeed of supply voltage
 | ⑥ | Dark-colored BaseUnits BUB for conducting the potential group further
 | ⑦ | BaseUnit for motor starters
 | ⑧ | Potential distributor module
 | ⑨ | Ex BaseUnit for Ex power module
 | ⑩ | Ex BaseUnit for Ex I/O module
 | ⑪ | Server module (included in the scope of supply of the CPU/interface module)
 | ⑫ | Ex I/O module
 | ⑬ | Ex power module
 | ⑭ | ET 200SP motor starter
 | ⑮ | I/O module

 | ① | Interface module
 | ② | Light-colored BaseUnit BUD with infeed of supply voltage
 | ③ | Dark-colored BaseUnits BUB for conducting the potential group further
 | ④ | I/O module
 | ⑤ | Server module (ships with the interface module)
 | ⑥ | Fail-safe I/O modules
 | ⑦ | BusAdapter
 | ⑧ | Mounting rail
 | ⑨ | Reference identification label

 | SIL2 | Category 3 | (PL) Performance Level d
 | --- | --- | ---
 | SIL3 | Category 

## Parsed Sections

In [7]:
for section in doc.sections():
    print(section.to_text())

Installation
7.1 Basics
Introduction
Installation location
Mounting position
Mounting rail
NOTE
NOTE
NOTE
Minimum clearances
NOTE Ex module group
General rules for installation
WARNING Hazardous Voltage Can Cause Death, Serious Injury, or Property Damage.
NOTE
WARNING Protection from conductive contamination
Mounting rules for reducing the thermal load
7.2 Installation conditions for motor starters
Mechanical brackets
Designing interference-free motor starters
Mount the dummy module
NOTICE Ensure interference immunity
7.3 Mounting the CPU/interface module
Introduction
Requirement
Required tools
Mounting the CPU/interface module
Dismantling the CPU/interface module
NOTE
7.4 Installing ET 200SP R1
Introduction
Requirement
Tools required
Mounting the ET 200SP R1 system


## Search for specific sections/tables in the document:
### Here Section: 5.2 What are fail-safe automation systems and fail-safe modules?

In [277]:
def get_section_text(doc, section_title):
    """
    Extracts the text from a specific section in a parsed PDF document.

    Parameters:
    - doc (Document): A Document object from the llmsherpa.readers.layout_reader library.
    - section_title (str): The title of the section to extract.

    Returns:
    - str: The HTML representation of the section's content, or a message if the section is not found.
    """

    selected_section = None

    # Find the desired section by title
    for section in doc.sections():
        if section.title == section_title:
            selected_section = section
            break

    # If the section is not found, return a message
    if not selected_section:
        return f"No section titled '{section_title}' found."

    # Return the full content of the section as HTML
    return selected_section.to_html(include_children=True, recurse=True)

In [278]:
section_text = get_section_text(doc, 'Mounting the CPU/interface module')
HTML(section_text)

## Create RAG

### Create ChromaDB
ChromaDB stores the vectorstore (embeddings) persistent on the disc to be re-used

In [267]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("MergedCollection")

### Setup LLM model and embeddings model

In [268]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

llm = Ollama(model="llama3", request_timeout=120.0)
Settings.llm = llm

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

In [269]:
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import Document

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


### Insert parsed document chunks into the vectorstore

### Insert chunk with metadata into the vectorstore index

In [84]:
# from llama_index.core.schema import MetadataMode

# # If there is no existing vector store create a new one and assign it to the index
# index = VectorStoreIndex([], storage_context=storage_context)

# documents = []
# for chunk_id, chunk in enumerate(doc.chunks()):
#     document = Document(text=chunk.to_context_text(), 
#                         id_=chunk_id,
#                         metadata={
#                               #"bock_idx": chunk.block_idx, # Not sure if needed
#                               "page_number": chunk.page_idx + 1, # We add 1 to the page index to match the actual page number
#                               # "tag": chunk.tag,
#                               "parent_section": chunk.parent.title,
#                               "parent_section_hierarchy": get_chunk_parent_titles(chunk)
#                               #,"hierarchy_level": chunk.level # Not sure if needed 
#                               },
#                         text_template="Metadata\n{metadata_str}\nContent:\n{content}",)
#     documents.append(document)
#     print_chunk_metadata(chunk)
                        
#     index.insert(document)

#     # # Print the context of the chunk for debugging
#     # if chunk_id < 10:
#     #     print("---------------------The LLM sees this:---------------------",)
#     #     print(document.get_content(metadata_mode=MetadataMode.LLM),)
#     #     print("---------------------The Embedding model sees this:---------------------",)
#     #     print(document.get_content(metadata_mode=MetadataMode.EMBED),)
#     #     print("="*100)


Page Nr:  1
Tag:  para
Parent Section:  SYSTEM MANUAL
Parent Section Hierarchy:  SYSTEM MANUAL
Level in the hierarchy:  1

Context Text:  SYSTEM MANUAL
11/2023Edition

Page Nr:  2
Tag:  para
Parent Section:  SIMATIC ET 200SP
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
Level in the hierarchy:  5

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.

Page Nr:  4
Tag:  para
Parent Section:  Area of application
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Level in the hierarchy:  6

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SI

### Insert custom merged chunks

In [271]:
from llama_index.core.schema import MetadataMode

# If there is no existing vector store create a new one and assign it to the index
index = VectorStoreIndex([], storage_context=storage_context)

documents = []
for chunk_id, chunk in enumerate(final_chunks):
    document = Document(text=chunk['text'], 
                        id_=chunk_id,
                        metadata={
                              #"bock_idx": chunk.block_idx, # Not sure if needed
                              "page_number": chunk['page_nr'], # We add 1 to the page index to match the actual page number
                              # "tag": chunk.tag,
                              "parent_section": chunk['parent'],
                              "parent_section_hierarchy": chunk['section']
                              #,"hierarchy_level": chunk['level'] # Not sure if needed 
                              },
                        text_template="Metadata\n{metadata_str}\nContent:\n{content}",)
    documents.append(document)
                        
    index.insert(document)

    # # Print the context of the chunk for debugging
    # if chunk_id < 10:
    #     print("---------------------The LLM sees this:---------------------",)
    #     print(document.get_content(metadata_mode=MetadataMode.LLM),)
    #     print("---------------------The Embedding model sees this:---------------------",)
    #     print(document.get_content(metadata_mode=MetadataMode.EMBED),)
    #     print("="*100)


Load existing vectorstore if one exists

In [15]:
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

### Query llm with context using Ollama Serve

Create prompt template based on LLAMA3 format

In [272]:
from llama_index.core import PromptTemplate

templateQA = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for answering questions about a technical manual. Use the following pieces of retrieved context from the manual to answer the question. \
If you don't know the answer, just say that you don't know. Be as detailed as possible.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Question: {query_str}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

# templateQA = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# You are a helper in creating assembly instructions based on a technical manual. Use the following pieces of retrieved context from the manual and the question to produce instructions. \
# If you don't know the answer, just say that you don't know. Be as detailed as possible.<|eot_id|>
# <|start_header_id|>user<|end_header_id|>

# Question: {query_str}
# Context: {context_str}
# Answer: <|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

templateSummary = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Context information from multiple sources is below. Given the information from multiple sources and not prior knowledge, answer the query.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Question: {query_str}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

templateRefine = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Based on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Original Question: {query_str}
Existing Answer: {existing_answer}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

qa_template = PromptTemplate(templateQA)
summary_template = PromptTemplate(templateSummary)
refine_template = PromptTemplate(templateRefine)

# you can create text prompt (for completion API)
prompt = refine_template.format(context_str=..., query_str=..., existing_answer=...)
print("Prompt:")
print(prompt)

print()
print("="*100)
print()

# or easily convert to message prompts (for chat API)
messages = refine_template.format_messages(context_str=..., query_str=..., existing_answer=...)
print("Messages:")
print(messages)

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Based on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Original Question: Ellipsis
Existing Answer: Ellipsis
Context: Ellipsis
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>


Messages:
[ChatMessage(role=<MessageRole.USER: 'user'>, content='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nBased on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nOriginal Question: Ellipsis\nExisting Answer: Ellipsis\nContext: Ellipsis\nAnswer: <|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>', additional_kwargs={})]


### Helper function for printing context nodes

In [273]:
def print_context(response):
    print("Nr of context nodes: ", len(response.source_nodes))
    print()
    print("="*150)
    for node in response.source_nodes:
        print("---------------------Metadat:---------------------")
        print(node.node.metadata)
        print("---------------------Text:---------------------")
        print(node.text)
        print("="*150)

## Customize Retrievers
### Based on Similarity
Uses top-k = 10 and similarity cutoff = 60% 

### TODO: Add own retrieval algorithm based on sections (Install, Mounting, Assembly, Disassembly, ...)
https://docs.llamaindex.ai/en/stable/understanding/querying/querying/

https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/

In [12]:
#A dummy node-postprocessor can be implemented in just a few lines of code:
from typing import List, Optional

from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class DummyNodePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:
        # subtracts 1 from the score
        for n in nodes:
            n.score -= 1

        return nodes

### Based on Keywords
Include and exclude required keywords from the context

In [13]:
# Helper function to view the prompt format
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} -> " f"**Text:**"
        print(text_md)
        print(p.get_template())

TODO: Explore different retrievers: https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide/

In [274]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor,KeywordNodePostprocessor, MetadataReplacementPostProcessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)


# configure response synthesizer: https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/
response_synthesizer = get_response_synthesizer(response_mode="compact") # response mode "refine" for more detailed responses

# This works with or logic, so if any of the required keywords are found, the node is kept
node_postprocessors = [
    # KeywordNodePostprocessor(
    #     required_keywords=["Basic component"]#, exclude_keywords=["Italy"]
    #     # required_keywords=[]
    # ),
    # SimilarityPostprocessor(similarity_cutoff=0.6)
    # MetadataReplacementPostProcessor(target_metadata_key="window")
]

#"default": "create and refine" an answer by sequentially going through each retrieved Node; This makes a separate LLM call per Node. Good for more detailed answers.
#"tree_summarize": Given a set of Node objects and the query, recursively construct a tree and return the root node as the response. Good for summarization purposes.
response_mode = "tree_summarize" 

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever, 
    response_synthesizer=response_synthesizer, 
    node_postprocessors=node_postprocessors, 
    # response_mode=response_mode
)

# Update RetrieverQueryEngine prompt template to match LLAMA3 format
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_template,
                            "response_synthesizer:summary_template": summary_template,
                            "response_synthesizer:refine_template": refine_template})

# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)

In [275]:
# query
response = query_engine.query("Provide me the Installation conditions for motor starters.")
print(response)

print()
print("Provided context: ")
print_context(response)

According to the manual, the installation conditions for motor starters are as follows:

1. Mounting position:
	* You can fit the motor starter vertically or horizontally.
	* The mounting position refers to the alignment of the mounting rail.
	* For a vertical mounting position, use end retainers "8WA1808" at both ends of the ET 200SP station.
2. Mounting rail:
	* Use one of the following mounting rails:
		+ 35x15 mm DIN rail in accordance with DIN EN 60715
		+ 35x7.5 mm DIN rail in accordance with DIN EN 60715
		+ SIMATIC S7 mounting rail
3. Current carrying capacity of the ET 200SP station:
	* Consider the current load via the power bus and the infeed bus of the ET 200SP station.
4. Ambient temperature range:
	* Up to 60°C: Horizontal mounting position
	* Up to 50°C: Vertical installation position
5. Mechanical brackets:
	* Use mechanical brackets in the following situations:
		+ When using a 15 mm mounting rail with a single motor starter installation, i.e., no motor starter mounted

In [144]:
for chunk in doc.chunks():
    print_chunk_metadata(chunk)
    print("="*100)

Page Nr:  1
Tag:  para
Parent Section:  SYSTEM MANUAL
Parent Section Hierarchy:  SYSTEM MANUAL
Level in the hierarchy:  1

Context Text:  SYSTEM MANUAL
11/2023Edition

Page Nr:  2
Tag:  para
Parent Section:  SIMATIC ET 200SP
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
Level in the hierarchy:  5

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.

Page Nr:  4
Tag:  para
Parent Section:  Area of application
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Level in the hierarchy:  6

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SI

In [16]:
# query
response = query_engine.query("I have two questions answer them separately. How do I mount a cpu module on the system rail and how do I dismantle a cpu module? In which section and on which page can I find this information?")
print(response)

print()
print("Actual context: ")
print_context(response)

To mount a CPU module on the system rail:

You can find this information in the "Mounting the CPU/interface module" section, page 7. According to the content, to install a CPU/interface module, follow these steps:
1. Install the CPU/interface module on the mounting rail.
2. Swivel the CPU/interface module towards the back until you hear the mounting rail release button click into place.

To dismantle a CPU module:

According to the provided context, to remove the CPU/interface module, follow these steps:
1. Switch off the supply voltage for the CPU/interface module.
2. Remove the 24 V DC connector from the CPU/interface module.
3. Press the mounting rail release button on the first BaseUnit.
At the same time, shift the CPU/interface module parallel to the left until it detaches from the rest of the module group.

Note: The information is found in the "Dismantling the CPU/interface module" section, page 8.

Actual context: 
Nr of context nodes:  7

---------------------Metadat:---------

In [88]:
# query
response = query_engine.query("Tell me the tools I need for the installation of ET 200SP R1.")
print(response)

print()
print("Actual context: ")
print_context(response)

According to the provided context, the tools you need for the installation of ET 200SP R1 are:

* 3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)

This information is found in the "Tools required" section on page 8 of the metadata.

Actual context: 
Nr of context nodes:  10

---------------------Metadat:---------------------
{'page_number': 8, 'tag': 'para', 'parent_section': 'Mounting the ET 200SP R1 system', 'parent_section_hierarchy': 'Installation > 7.4 Installing ET 200SP R1 > Mounting the ET 200SP R1 system'}
---------------------Text:---------------------
Installation > 7.4 Installing ET 200SP R1 > Mounting the ET 200SP R1 system
To mount the ET 200SP R1 system, proceed as follows:
1. Hang the BaseUnit BU type M0 onto the SIMATIC system rail.
2. Swivel the BaseUnit BU type M0 backwards until the system rail release audibly engages.
3. Plug the IM 155-6 PN R1 interface modules onto the BaseUnit BU type M0 until the lock audibly engages.
4. Plug the 24 V DC

### TODO: Explore structured output
https://docs.llamaindex.ai/en/stable/module_guides/querying/structured_outputs/query_engine/

Allows for parsing the llm output into an object like Manual entry: string section, string text

### TODO: Test llamaparse for PDF parsing
https://docs.llamaindex.ai/en/stable/module_guides/loading/connector/llama_parse/

--> DONE: Parsing and indexing is superior to llmsherpa out of the box but requires payed API key

### TODO: Benchmarking


- Retrieval Evaluation: This assesses the accuracy and relevance of the information retrieved by the system.
- Response Evaluation: This measures the quality and appropriateness of the responses generated by the system based on the retrieved information.


In [33]:
def add_qa_to_list(lst, question, answer):
    lst.append({question: answer})
    return lst

qa_list = []
add_qa_to_list(qa_list, "What is the first step in mounting the CPU/interface module?", "Install the CPU/interface module on the mounting rail.")
add_qa_to_list(qa_list, "What tools are needed for the installation of ET 200SP R1?", "3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)")
add_qa_to_list(qa_list, "What are the requirements for installing a ET 200SP R1??", "The SIMATIC system rail is installed.")
add_qa_to_list(qa_list, "Which type of mounting rails can I use for motor starters?", "– 35x15 mm DIN rail in accordance with DIN EN 60715\n– 35x7.5 mm DIN rail in accordance with DIN EN 60715\n– SIMATIC S7 mounting rail")


[{'What is the first step in mounting the CPU/interface module?': 'Install the CPU/interface module on the mounting rail.'},
 {'What tools are needed for the installation of ET 200SP R1?': '3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)'},
 {'What are the requirements for installing a ET 200SP R1??': 'The SIMATIC system rail is installed.'},
 {'Which type of mounting rails can I use for motor starters?': '– 35x15 mm DIN rail in accordance with DIN EN 60715\n– 35x7.5 mm DIN rail in accordance with DIN EN 60715\n– SIMATIC S7 mounting rail'}]

In [39]:
def get_context_string(response):
    string = ""
    for node in response.source_nodes:
        string += f"Context Metadata:\n"
        string += f"{node.node.metadata}\n"
        string += f"Context Text:\n"
        string += f"{node.text}\n"
        string += "\n\n"
    return string

In [53]:
templateEvalContextRelevance = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Your task is to evaluate if the provided context information is sufficient to answer a question correctly. \
You are provided with the context and the expected response. \n
You have two options to answer. Either YES/ NO. \n
Answer - YES, if all the information in the expected response are included in the context otherwise NO.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Context: \n {context_str}\n
Expected Response: \n {expected_response}\n
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

eval_template = PromptTemplate(templateEvalContextRelevance)

In [58]:
eval_index = VectorStoreIndex([])
eval_query_engine = eval_index.as_chat_engine()

def execute_evaluation(qa_entry):
    query = list(qa_entry.keys())[0]
    expected_repsonse = list(qa_entry.values())[0]

    # Execute query with question
    response = query_engine.query(query)

    # Get retrieved context
    context = get_context_string(response)
    expected_repsonse = expected_repsonse

    # Print everything for debugging
    print("--------------------------Query: --------------------------")
    print(query)
    print("--------------------------Response: --------------------------")
    print(response)
    print("--------------------------Context: --------------------------")
    print(context)
    print("--------------------------Expected Response: --------------------------")
    print(expected_repsonse)

    # Generate evaluation prompt
    eval_prompt = eval_template.format(context_str=context, expected_response=expected_repsonse)

    # Evaluate the context relevance
    eval_response = eval_query_engine.query(eval_prompt)

    print("--------------------------Evaluation : --------------------------")
    print(eval_response)

    return eval_response

In [59]:
execute_evaluation(qa_list[1])

--------------------------Query: --------------------------
What tools are needed for the installation of ET 200SP R1?
--------------------------Response: --------------------------
The typical tools and equipment needed for installing Siemens ET 200SP R1 compact controllers include screwdrivers, Allen wrenches, wire strippers, crimping tools, and a multimeter. Additionally, a programming device such as a laptop with the corresponding software may be required for configuration purposes.
--------------------------Context: --------------------------

--------------------------Expected Response: --------------------------
3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)
--------------------------Evaluation : --------------------------
You would need a 3 to 3.5 mm screwdriver for mounting and removing the BusAdapter.


Response(response='You would need a 3 to 3.5 mm screwdriver for mounting and removing the BusAdapter.', source_nodes=[], metadata=None)